<a href="https://colab.research.google.com/github/PauloAAlmeida/Python/blob/master/Rede_Neurais_sonar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neurais --------------------------------

### Base: Sonar, Mines vs. Rocks

https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar,+Mines+vs.+Rocks%29






In [10]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

np.set_printoptions(precision=2)



### Carga dos dados


In [11]:
from google.colab import files
uploaded = files.upload()
sonar = pd.read_excel('sonar.all-data.xlsx', sheet_name=0) 
from google.colab import drive




Saving sonar.all-data to sonar.all-data


In [12]:
X = sonar.iloc[:,0:(sonar.shape[1] - 1)]

le = LabelEncoder()
y = le.fit_transform(sonar.iloc[:,(sonar.shape[1] - 1)])

class_names = le.classes_


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)



### MLP com uma camada oculta

O bloco a seguir testa a rede neural com um camada oculta com 100 neurônios. 

São totalizados 6.100 pesos diferentes que precisarão ser ajustados na fase de treinamento.

O parâmetro solver='lbfgs' foi escolhido por ser mais adequado para treinamento com bases pequenas (menores que alguns milhares de registros).

In [13]:
# Rede Perceptron Multicamadas (MLP):  Configuração default otimizando a função log-loss
# uma camada oculta com 100 neurônios.

mlp = MLPClassifier(solver='lbfgs', random_state=0)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)


print("Camadas da rede: {}".format(mlp.n_layers_))
print("Neurônios na camada oculta: {}".format(mlp.hidden_layer_sizes))
print("Neurônios na camada de saída: {}".format(mlp.n_outputs_))
print("Pesos na camada de entrada: {}".format(mlp.coefs_[0].shape))
print("Pesos na camada oculta: {}".format(mlp.coefs_[1].shape))

print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test, y_test)))


print(classification_report(y_test, y_pred, target_names=class_names))

# Calcula a matriz de confusão
cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)

Camadas da rede: 3
Neurônios na camada oculta: (100,)
Neurônios na camada de saída: 1
Pesos na camada de entrada: (60, 100)
Pesos na camada oculta: (100, 1)
Acurácia da base de treinamento: 0.53
Acurácia da base de teste: 0.62
              precision    recall  f1-score   support

           M       0.60      0.33      0.43         9
           R       0.62      0.83      0.71        12

    accuracy                           0.62        21
   macro avg       0.61      0.58      0.57        21
weighted avg       0.61      0.62      0.59        21

[[ 3  6]
 [ 2 10]]


### MLP com duas camadas ocultas

O bloco a seguir testa a rede neural com duas camadas ocultas. 

A primeira camada possui 100 neurônios, enquanto a segunda camada possui 60 neurônios. 

São totalizados 12.100 pesos diferentes que precisarão ser ajustados na fase de treinamento.

Com essa rede será possível observar que aumentar arbitrariamente a dimensão da sua rede neural não garante um aumento arbitrário da performance do modelo.

In [14]:

mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[100, 60])
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)


print("Camadas da rede: {}".format(mlp.n_layers_))
print("Neurônios na camada oculta: {}".format(mlp.hidden_layer_sizes))
print("Neurônios na camada de saída: {}".format(mlp.n_outputs_))
print("Pesos na camada de entrada: {}".format(mlp.coefs_[0].shape))
print("Pesos na camada oculta: {}".format(mlp.coefs_[1].shape))

print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test, y_test)))

print(classification_report(y_test, y_pred, target_names=class_names))

cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)


Camadas da rede: 4
Neurônios na camada oculta: [100, 60]
Neurônios na camada de saída: 1
Pesos na camada de entrada: (60, 100)
Pesos na camada oculta: (100, 60)
Acurácia da base de treinamento: 0.58
Acurácia da base de teste: 0.57
              precision    recall  f1-score   support

           M       0.50      0.22      0.31         9
           R       0.59      0.83      0.69        12

    accuracy                           0.57        21
   macro avg       0.54      0.53      0.50        21
weighted avg       0.55      0.57      0.53        21

[[ 2  7]
 [ 2 10]]


### *Overfitting* por excesso de neurônios

A forma mais eficiente para se determinar o número de neurônios na camada oculta é por busca sistemática. Um artigo interessante que ilustra diversas heurísticas para resolver o problema pode ser visto em 

 D. Stathakis (2009) *How many hidden layers and nodes?*, **International Journal of Remote Sensing**, 30:8, 2133-2147, DOI: 10.1080/01431160802549278 
 
 Entretanto, um ponto de partida inicial muito utilizado corresponde ao:
 
 (num_entradas + num_saídas) / 2.
 
 Note que a rede manteve a mesma performance das topologias anteriores.

In [16]:
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[31])
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)


print("Camadas da rede: {}".format(mlp.n_layers_))
print("Neurônios na camada oculta: {}".format(mlp.hidden_layer_sizes))
print("Neurônios na camada de saída: {}".format(mlp.n_outputs_))
print("Pesos na camada de entrada: {}".format(mlp.coefs_[0].shape))
print("Pesos na camada oculta: {}".format(mlp.coefs_[1].shape))

print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test, y_test)))

print(classification_report(y_test, y_pred, target_names=class_names))

cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)


Camadas da rede: 3
Neurônios na camada oculta: [31]
Neurônios na camada de saída: 1
Pesos na camada de entrada: (60, 31)
Pesos na camada oculta: (31, 1)
Acurácia da base de treinamento: 0.58
Acurácia da base de teste: 0.48
              precision    recall  f1-score   support

           M       0.44      0.89      0.59         9
           R       0.67      0.17      0.27        12

    accuracy                           0.48        21
   macro avg       0.56      0.53      0.43        21
weighted avg       0.57      0.48      0.41        21

[[ 8  1]
 [10  2]]


### Ajustamento dos dados

As redes neurais a seguir irão testar a hipótese de que uma MLP é robusta quanto a dados não normalizados. 

Os dados serão padronizados pelo Z-score.

A normalização dos dados não irá acarretar em nenhume melhoria da rede.

In [17]:
# Calcula a média e o desvio padrão de cada atributo da base de treinamento
mean_on_train = X_train.mean(axis=0)
std_on_train = X_train.std(axis=0)

# Normaliza os atributos pela norma Z = (X - média) / desvio padrão
# afterwards, mean=0 and std=1
X_train_scaled = (X_train - mean_on_train) / std_on_train
# usa a esma transformação nos dados de teste
X_test_scaled = (X_test - mean_on_train) / std_on_train


# A rede atinge o número máximo de iterações, mas não converge.
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=[31], random_state=0)
mlp.fit(X_train_scaled, y_train)
print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train_scaled, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test_scaled, y_test)))

# Vamos aumentar o número máximo de iterações
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=[31], max_iter=1000, random_state=0)
mlp.fit(X_train_scaled, y_train)
print("Acurácia da base de treinamento: {:.2f}".format(mlp.score(X_train_scaled, y_train)))
print("Acurácia da base de teste: {:.2f}".format(mlp.score(X_test_scaled, y_test)))


Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.67
Acurácia da base de treinamento: 1.00
Acurácia da base de teste: 0.67
